`comparaison_env`

In [13]:
import pandas as pd
import glob
import os 
import sys
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import random
import tqdm
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import entropy, pearsonr
from scipy.spatial.distance import jensenshannon

sys.path.append(os.path.abspath("/home/pelissier/These-ATER/Papier_international3/Dataset"))
from utils import *
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Code')  # Adjust the path based on the relative location
from utils_comparaison import *
from metriques import *

Variables globales

In [14]:
### MODELNET40 REMESHING ISO
ModelNet40_aligned_us = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"
# Data des 12 caméras du Rendu
data_modelnet_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/circular_config_12_elevation_30_R22.csv")
# Path ok (avec limper + projection + bvs)
paths_bvs = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt"); print("Fichiers bvs de Modelnet40 :", len(paths_bvs))
dir_bvs = "/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/bvs_remeshing_iso"

##################################################################################################################################
### User study
# Path to the directory containing the csv files of the user study
dir_us = "/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific"
# Data of camera poses in user study : i, j, theta, delta, X, Y, Z
data_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_pose_rep_etude_arrondi.csv")
label_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_rep_etude_label_arrondi.csv")
data_us_cam['label'] = list(label_us_cam['label'])
## Choix et bvs des 44 modeles 
dir_bvs_us = '/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific/csv_etude_filtre/visualisation_filtre'
paths_bvs_us_csv = glob.glob(os.path.join(dir_bvs_us, "**", "*global_distribution_label.csv*"), recursive=True); print("Modeles de US : ",len(paths_bvs_us_csv))
# Path data folder of user study
dir_Data = "/home/pelissier/These-ATER/Papier_internationale2/Data"

##################################################################################################################################
# Correspondances entre les noms des modèles dans ModelNet10 et les noms des modèles dans l'User Study
match_ModelNet2US = {'airplane': 'A380', "chair":'chair107', 'bench': 'bench', 'dresser': 'cabinet-d', 'night_stand': 'cabinet-n', 'tv_stand': 'cabinet-t', 'vase': 'vase', 'cup':'cup', 'car': 'carVasa'}
# Outputs tmp
# Path of user-study outputs folder in Dataset
dir_outputs_visu = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/visualisation_cams/"
dir_outputs_csv = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/csv_files/"

Fichiers bvs de Modelnet40 : 844
Modeles de US :  44


In [15]:
list_categorie_us = ['carVasa', 'cup', "A380", "cabinet-d", "cabinet-t", "bench", "chair107", "cabinet-n", "vase"]
list_categroie_us_sym = ['carVasa', 'cup', "A380", "cabinet-d", "cabinet-t", "bench", "cabinet-n", "vase"]

## Symetrie considérée
## Tous les objets peut importe le nb d'axe de symétrie
dir_outputs_visu = os.path.join(dir_outputs_visu, "All/")
dir_outputs_csv = os.path.join(dir_outputs_csv, "All/")

## Contraintes : quelles caméras de l'US on considère ?

In [16]:
## Condition
## Toutes les caméras 
#j_cam_us_ok = [0,1,2,3,4]

## Que les cameras de la couronne j==1
#j_cam_us_ok = [1]; dir_outputs_visu = os.path.join(dir_outputs_visu, "8cams/"); dir_outputs_csv = os.path.join(dir_outputs_csv, "8cams/")

## Que les cameras de la couronne j==1 ou j == 2
j_cam_us_ok = [1,2]; dir_outputs_visu = os.path.join(dir_outputs_visu, "16cams/"); dir_outputs_csv = os.path.join(dir_outputs_csv, "16cams/")

## Cameras

In [17]:
## User study
# Coordonnées des caméras de l'étude utilateur
X_us = []; Y_us = []; Z_us = []; labels_us = []; I_us = []; J_us = []
for j in j_cam_us_ok:
    X_us += list(data_us_cam.loc[data_us_cam['j'] == j]['X_rep_etude'])
    Y_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Y_rep_etude'])
    Z_us += list(data_us_cam.loc[data_us_cam['j'] == j]['Z_rep_etude'])
    I_us += list(data_us_cam.loc[data_us_cam['j'] == j]['i'])
    J_us += [j for _ in range(8)]
    labels_us += list(data_us_cam.loc[data_us_cam['j'] == j]['label']) 
cams_us = np.around(np.column_stack((X_us, Y_us, Z_us, np.array([1]*len(X_us)))),3)
R_sphere = list(data_us_cam['R'])[0]

## ModeleNet 
# Coordonnées des 12 caméras de ModelNet40 dans le repère de ModelNet40
X_modelnet = np.array(data_modelnet_cam['LocationX'][1:])
Y_modelnet = np.array(data_modelnet_cam['LocationY'][1:])
Z_modelnet = np.array(data_modelnet_cam['LocationZ'][1:])
cams_modelnet = np.column_stack((X_modelnet, Y_modelnet, Z_modelnet, np.array([1]*12)))
print("12 cams Modelnet dans repère Modelnet : \n", cams_modelnet)

12 cams Modelnet dans repère Modelnet : 
 [[ 0.         -2.2         1.1         1.        ]
 [ 1.1        -1.90525589  1.1         1.        ]
 [ 1.90525589 -1.1         1.1         1.        ]
 [ 2.2         0.          1.1         1.        ]
 [ 1.90525589  1.1         1.1         1.        ]
 [ 1.1         1.90525589  1.1         1.        ]
 [ 0.          2.2         1.1         1.        ]
 [-1.1         1.90525589  1.1         1.        ]
 [-1.90525589  1.1         1.1         1.        ]
 [-2.2         0.          1.1         1.        ]
 [-1.90525589 -1.1         1.1         1.        ]
 [-1.1        -1.90525589  1.1         1.        ]]


In [18]:
print(len(cams_us), "cams US", len(labels_us), "lables US")
print(str(len(cams_us))+" cams US considérées dans repère US : \n", cams_us, labels_us)

16 cams US 16 lables US
16 cams US considérées dans repère US : 
 [[ 1.56  1.56  0.    1.  ]
 [ 1.1   1.56  1.1   1.  ]
 [ 0.    1.56  1.56  1.  ]
 [-1.1   1.56  1.1   1.  ]
 [-1.56  1.56  0.    1.  ]
 [-1.1   1.56 -1.1   1.  ]
 [-0.    1.56 -1.56  1.  ]
 [ 1.1   1.56 -1.1   1.  ]
 [ 2.2   0.    0.    1.  ]
 [ 1.56  0.    1.56  1.  ]
 [ 0.    0.    2.2   1.  ]
 [-1.56  0.    1.56  1.  ]
 [-2.2   0.    0.    1.  ]
 [-1.56  0.   -1.56  1.  ]
 [-0.    0.   -2.2   1.  ]
 [ 1.56  0.   -1.56  1.  ]] ['dessus_face', 'dessus_face_droit', 'dessus_profil_droit', 'dessus_arriere_droit', 'dessus_arriere', 'dessus_arriere_gauche', 'dessus_profil_gauche', 'dessus_face_gauche', 'face', 'face_droit', 'profil_droit', 'arriere_droit', 'arriere', 'arriere_gauche', 'profil_gauche', 'face_gauche']


## Poids + BVS user study --> en fonction des caméras de l'us

In [19]:
all_bvs_us = {}
# Les POV qui nous intéressent sont ceux sur la couronne 'Milieu-Dessus', donc avec j == 1
for path_bvs_us_csv in paths_bvs_us_csv:
    name = os.path.basename(path_bvs_us_csv).split('_')[0]
    df = pd.read_csv(path_bvs_us_csv)
    # on ne garde que les info de caméra concidérées
    filter_df = df[df['label'].isin(labels_us)]
    filter_sorted_df = filter_df.sort_values(by='poids', ascending=False)
    # BVS : attention il put y avoir plusieurs BVS pour un même modèle <=> plusieurs labels avec le même poids
    label_bvs_mesh = list(filter_sorted_df.loc[filter_sorted_df['poids'] ==  max(list(filter_sorted_df['poids']))]['label'])
    position_bvs_mesh = [[i for i in range(len(labels_us)) if labels_us[i] in label_bvs_mesh]]
    # coordonnées 3D des caméras BVS dans le repère de l'US
    cam_bvs = np.concatenate([np.array(X_us)[position_bvs_mesh], np.array(Y_us)[position_bvs_mesh], np.array(Z_us)[position_bvs_mesh]]).T  
    # Indice I-J des caméras dans le repère de l'US
    i_j_cam = np.concatenate([np.array(I_us)[position_bvs_mesh], np.array(J_us)[position_bvs_mesh]]).T
    all_bvs_us[name] = {'df': filter_sorted_df, 'label_bvs': label_bvs_mesh, 'cam_bvs': cam_bvs, 'ij_bvs': i_j_cam, 'poids_bvs' :  max(list(filter_sorted_df['poids']))}

# Relancer partie suivante quand tous les bvs seront calculés

## Pour 1 categorie : cvs avec poids par caméras US

In [20]:
match_ModelNet2US.keys()

dict_keys(['airplane', 'chair', 'bench', 'dresser', 'night_stand', 'tv_stand', 'vase', 'cup', 'car'])

In [21]:
categorie_modelnet = "car"
categorie_us = match_ModelNet2US[categorie_modelnet]; print(categorie_modelnet, categorie_us)
if 'cabinet' in categorie_us: categorie_us = 'cabinet'
print(categorie_modelnet, categorie_us)

car carVasa
car carVasa


In [22]:
## Modele de l'US asscoié à la catégorie
path_mesh_us = os.path.join(dir_Data, categorie_us, categorie_us+"_update_normed_centered_user_study.obj")
#path_mesh_us = os.path.join(dir_Data, 'mesh_objectnet_regulier', categorie_us, categorie_us+"_regulier_tri_normed_centered_user_study.obj")
print(os.path.exists(path_mesh_us))

# dossier pour les visualisations
if not os.path.exists(os.path.join(dir_outputs_visu, categorie_modelnet)):
    os.makedirs(os.path.join(dir_outputs_visu, categorie_modelnet), exist_ok=True)
dir_outputs_visu_categorie = os.path.join(dir_outputs_visu, categorie_modelnet); print(dir_outputs_visu_categorie)

True
/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/visualisation_cams/All/16cams/car


### Impact/Poids de la BVS de modelnet aligné sur les X cams US --> BVS sur l'ensemble des modèles 

In [23]:
## Fichiers BVS de la categorie
paths_bvs = [paths_bvs[i] for i in range(len(paths_bvs)) if categorie_modelnet in paths_bvs[i]]; print("Fichiers bvs de la categorie dispo :", len(paths_bvs), paths_bvs[0])
random.shuffle(paths_bvs)
print(paths_bvs[0])

Fichiers bvs de la categorie dispo : 38 car/train/car_0166_SMPLER_centered_scaled_remeshing_iso_iter4
car/test/car_0283_SMPLER_centered_scaled_remeshing_iso_iter3


Création des fichiers CAT_distribution_global_modelnet-Xcams.csv

In [24]:
all_bvs_modelnet = {}
## Df des poids de chaque mesh de la catégorie sur les caméras de l'US
df_poids_from_modelnet = pd.DataFrame(columns=['path', 'name']+[f"{int(I_us[k])}-{J_us[k]}" for k in range(len(I_us))])
df_poids_from_modelnet.loc[0] = [None, 'labels US'] + labels_us

if False:
    # Pour chacun des modèles de la catégorie étudée dont on a le fichier bvs (que 38 actuellement), issue de ModelNet40 (car : 296)
    for n in tqdm.tqdm(range(len(paths_bvs))):
        ###################################################################
        ## ModelNet40
        # Load model alignés à l'étude utilisateur
        path_mesh_n = paths_bvs[n]; #print(path_mesh_n)
        #path_mesh_n = "cup/train/cup_0039_SMPLER_centered_scaled_remeshing_iso_iter9"
        name_modelnet_n =  '_'.join(os.path.basename(path_mesh_n).split('_')[:3])
        df_poids_from_modelnet.loc[n+1, 'path'] = path_mesh_n; df_poids_from_modelnet.loc[n+1, 'name'] = name_modelnet_n
        # Load mesh PRÉALABLEMENT placé dans le REPRÈRE US (avec code Alignenment/align_mesh.ipynb)
        path_mesh_modelnet_aligned_n = os.path.join(ModelNet40_aligned_us, path_mesh_n+"_aligned_ok_US.obj")
        mesh_modelnet_aligned_n = trimesh.load_mesh(path_mesh_modelnet_aligned_n)
        centroid_modelnet_aligned_n = get_centroid(mesh_modelnet_aligned_n.faces, mesh_modelnet_aligned_n.vertices)
        # BVS du mesh ModelNet40 aligned
        cams_modelnet_mesh_n, cam_bvs_modelnet_n, num_cam_bvs_modelnet_n = bvs_cams_modelnet_aligned(path_mesh_n, path_mesh_modelnet_aligned_n, dir_bvs, cams_modelnet); #print("Modelnet", cam_bvs_modelnet_n, num_cam_bvs_modelnet_n)
        
        ## Objectif : trouver la "BVS moyenne"  <--> attribué un poids d'impact a chacun des cam de l'US
        # Impact de la camera BVS de modlenet40 sur les caméras de l'étude utilisateur
        df_poids_from_modelnet, cam_sphere = poids_modelnet_sur_US(df_poids_from_modelnet, cam_bvs_modelnet_n, centroid_modelnet_aligned_n, cams_us, R_sphere, I_us, J_us)

        ## OBJ : caméras modelent40 alignées avec US
        show_cams(mesh_modelnet_aligned_n, np.vstack((cam_bvs_modelnet_n, cam_sphere)), name_modelnet_n+"_sphere", None, None, None, dir_outputs_visu_categorie, US_obj=False)

        ###################################################################
        ## User stuy
        # mesh random from User_study
        mesh_us = trimesh.load_mesh(path_mesh_us)
        # BVS US
        cam_bvs_us = all_bvs_us[categorie_us]['cam_bvs']
        ## Double obj : len(cams_us) cameras US et 12 caméras modelent40 alignées avec US
        show_cams(mesh_modelnet_aligned_n, cams_modelnet_mesh_n, name_modelnet_n, mesh_us, cams_us, categorie_us,  dir_outputs_visu_categorie, US_obj=True)  
        
        
    ## impact de chaque mesh de la catégorie sur les caméras de l'US
    df_poids_from_modelnet.to_csv(os.path.join(dir_outputs_csv, categorie_modelnet+"_distribution_impact-"+str(len(X_us))+"cams.csv"))
        
    ###################################################################
    ## Normalisation  des poids de modelnet sur US 
    # Somme des poids pour chaque caméras
    verif = 0
    df_poids_from_modelnet_final = pd.DataFrame(columns=['label', 'poids'])
    for k in range(len(I_us)):
        label_poids_k = list(df_poids_from_modelnet.loc[:, f"{int(I_us[k])}-{J_us[k]}"])
        df_poids_from_modelnet_final.loc[k] = [label_poids_k[0], np.sum(label_poids_k[1:])] 
        verif += np.sum(label_poids_k[1:])
        
    ## verification
    if float(abs(verif - len(paths_bvs))<10e-2): print("Somme des poids OK")
    else: print("Erreur de somme des poids")

    ###################################################################   
    # ## BVS de la catégorie courante
    df_poids_from_modelnet_final_sorted = df_poids_from_modelnet_final.sort_values(by='poids', ascending=False)
    # # sauvegarde
    df_poids_from_modelnet_final_sorted.to_csv(os.path.join(dir_outputs_csv, categorie_modelnet+"_distribution_global_modelnet-"+str(len(X_us))+"cams.csv"))
    df_poids_from_modelnet_final_sorted

## Comparaison

In [25]:
#list_categorie_us = ['vase']

### Data des bvs de US et Modelnet pour chaque catégorie

In [26]:
# Dicitonnaire : keys == categorie et values == info sur les BVS : cam, label, ij ....
BVS = {}
for cat_us in list_categorie_us :
    cat_m = [k for k, v in match_ModelNet2US.items() if v == cat_us][0]
    print(cat_m, cat_us)
    #categorie = categorie_us; print(categorie)

    ## bvs modelnet 
    df_bvs_modelnet = pd.read_csv(os.path.join(dir_outputs_csv, cat_m+"_distribution_global_modelnet-"+str(len(j_cam_us_ok)*8)+"cams.csv"))
    label_bvs_modelnet = list(df_bvs_modelnet.loc[df_bvs_modelnet['poids'] ==  max(list(df_bvs_modelnet['poids']))]['label'])
    position_bvs_modelenet = [[i for i in range(len(labels_us)) if labels_us[i] in label_bvs_modelnet]]
    # coordonnées 3D des caméras BVS dans le repère de l'US
    cam_bvs_modelnet = np.concatenate([np.array(X_us)[position_bvs_modelenet], np.array(Y_us)[position_bvs_modelenet], np.array(Z_us)[position_bvs_modelenet]]).T  
    # Indice I-J des caméras dans le repère de l'US
    ij_bvs_modelnet = np.concatenate([np.array(I_us)[position_bvs_modelenet], np.array(J_us)[position_bvs_modelenet]]).T
    #print('BVS modelenet dans repère US :\n', cam_bvs_modelnet, label_bvs_modelnet, ij_bvs_modelnet,"\n")

    ## bvs US
    cam_bvs_us = all_bvs_us[['cabinet' if 'cabinet' in cat_us else cat_us][0]]['cam_bvs']
    ij_bvs_us =  all_bvs_us[['cabinet' if 'cabinet' in cat_us else cat_us][0]]['ij_bvs']
    #print('US : \n', cam_bvs_us, all_bvs_us[cat_us]['label_bvs'], all_bvs_us[cat_us]['ij_bvs'])

    # Sauvegarde 
    BVS[cat_us] = {'modelnet': 
        {'cam': cam_bvs_modelnet, 
         'label': label_bvs_modelnet,
         'ij': ij_bvs_modelnet, 
         'df' : df_bvs_modelnet}, 
        'US': 
        {'cam': cam_bvs_us,
         'label': all_bvs_us[['cabinet' if 'cabinet' in cat_us else cat_us][0]]['label_bvs'], 
         'ij': all_bvs_us[['cabinet' if 'cabinet' in cat_us else cat_us][0]]['ij_bvs'], 
         'poids': all_bvs_us[['cabinet' if 'cabinet' in cat_us else cat_us][0]]['poids_bvs'], 
         'df' : all_bvs_us[['cabinet' if 'cabinet' in cat_us else cat_us][0]]['df']}, 'metriques': {}, 'metriques_sym': {}}
BVS.keys()

car carVasa
cup cup
airplane A380
dresser cabinet-d
tv_stand cabinet-t
bench bench
chair chair107
night_stand cabinet-n
vase vase


dict_keys(['carVasa', 'cup', 'A380', 'cabinet-d', 'cabinet-t', 'bench', 'chair107', 'cabinet-n', 'vase'])

### PS est le meme avec ou sans symetrique car on les prends déjà en compte dans la formule

In [27]:
# ce que je prédis :
print(get_ds2(coord_U= [1.10,1.56,1.1], coord_M=[0, 1.56, 1.56], sig=1.5, epsilon=2), 19.4/34.1)


for cat_us in list_categorie_us:
    print(cat_us)
    # Proximity score : Magenta : US // Sym uS : Bleu // Modelnet : Vert
    PS, terme = score_proximite(BVS, cat_us, path_mesh_us, list_categroie_us_sym, data_us_cam, dir_outputs_visu_categorie, sig =1.5, epsilon=2); print(PS)
    BVS[cat_us]['metriques']['PS'] = (np.round(PS,3), terme)
    BVS[cat_us]['metriques_sym']['PS'] = (np.round(PS,3), terme)

0.73 0.5689149560117301
carVasa
pov_u [0.  0.  2.2] [2. 2.]
pov_m [-0.    1.56 -1.56] [6. 1.]
cat sym
pov a un sym
sym [6, 2] [-0.   0.  -2.2]
mais pas les memes
Ds 0.53 - W 0.12857142857142856
0.53
cup
pov_u [0.  0.  2.2] [2. 2.]
pov_m [-1.1   1.56 -1.1 ] [5. 1.]
cat sym
pov a un sym
sym [6, 2] [-0.   0.  -2.2]
mais pas les memes
Ds 0.34 - W 0.4041666666666666
0.4041666666666666
A380
pov_u [1.1  1.56 1.1 ] [1. 1.]
pov_m [0.   1.56 1.56] [2. 1.]
cat sym
pov a un sym
sym [7, 1] [ 1.1   1.56 -1.1 ]
mais pas les memes
Ds 0.73 - W 0.35793357933579334
0.73
cabinet-d
pov_u [2.2 0.  0. ] [0. 2.]
pov_m [1.1  1.56 1.1 ] [1. 1.]
cat sym
pov n'a pas de sym
Ds 0.34 - W 0.061855670103092786
0.34
cabinet-t
pov_u [2.2 0.  0. ] [0. 2.]
pov_m [1.56 1.56 0.  ] [0. 1.]
cat sym
pov n'a pas de sym
Ds 0.53 - W 0.04639175257731959
0.53
bench
pov_u [1.56 1.56 0.  ] [0. 1.]
pov_m [1.56 1.56 0.  ] [0. 1.]
cool c'est le meme pov
Ds 1 - W 1.0
1
chair107
pov_u [2.2 0.  0. ] [0. 2.]
pov_m [1.1  1.56 1.1 ] [1. 1.]
c

# Pour normaliser les histogrammes (avec ou sans symétriques), on divise chacun par sa somme, comme ça la somme vaut 1 et les histogrammes peuvent être assimilés à des densité de probabilité discretes

### Avec symétriques

In [28]:
df_exemple, _ = get_poids_from_BVS(BVS, cat_us, labels_us, data_us_cam); df_exemple

,cat,label,poids_modelnet,poids_us,poids_modelnet_norm,poids_us_norm
0,vase,dessus_face,9.20,1.7,0.059,0.008
1,vase,dessus_face_droit,15.28,2.4,0.097,0.011
2,vase,dessus_profil_droit,18.86,12.0,0.120,0.057
3,vase,dessus_arriere_droit,15.27,7.5,0.097,0.035
4,vase,dessus_arriere,13.31,4.9,0.085,0.023
5,vase,dessus_arriere_gauche,13.90,2.7,0.089,0.013
6,vase,dessus_profil_gauche,19.72,5.0,0.126,0.024
7,vase,dessus_face_gauche,7.96,0.8,0.051,0.004
8,vase,face,5.70,9.0,0.036,0.042
9,vase,face_droit,6.72,10.0,0.043,0.047


In [17]:
dict_metriques = {"Dnom" : lambda w1, w2 : Dnom(w1, w2),
                  "Dord" :  lambda w1, w2 : Dord(w1, w2),
                  "Dmod" :  lambda w1, w2 : Dmod(w1, w2),
                  "L2_norm" : lambda w1, w2 : np.linalg.norm(w1 - w2),
                  "L1_norm" : lambda w1, w2 : np.sum(np.abs(w1 - w2)),
                  "cos_sim" : lambda w1, w2 : 1 - cosine(w1, w2),
                  "fourier_dist" : lambda w1, w2 : fourier_similarity(w1, w2),
                  "corr-pv" : lambda w1, w2 : pearsonr(w1, w2),
                  "kl_div" : lambda w1, w2 : entropy(w2, w1),
                  "js_div" : lambda w1, w2 : jensenshannon(w1, w2),
                  "bhatta_dist" : lambda w1, w2 : -np.log(np.sum(np.sqrt(np.array(w1) * np.array(w2)))), 
                  "circu_dist" : lambda w1, w2 : np.sum(np.minimum(np.abs(w1 - w2), 1 - np.abs(w1 - w2)))}

In [ ]:
for cat_us in list_categorie_us:
    cat_m = [k for k, v in match_ModelNet2US.items() if v == cat_us][0]
    df_poids_both, _ = get_poids_from_BVS(BVS, cat_us, labels_us, data_us_cam)
    # sauvegarde poids 
    df_poids_both.to_csv(os.path.join(dir_outputs_csv, cat_us+"_poids_both.csv"))
    # Poids des caméras pour les deux méthodes (exemple)
    weights_method_us = np.array(df_poids_both['poids_us_norm'])
    weights_method_m = np.array(df_poids_both['poids_modelnet_norm'])
    ####### Visualisation des poids 
    cams = [list(data_us_cam.loc[data_us_cam['label'] == l].iloc[0])[6:9] for l in df_poids_both['label']]
    # pour avoir de jolies couleurs il faut que les poids soient entre 0 et 1
    weights_histo_us = [x / max(weights_method_us) for x in weights_method_us]
    show_cams_histogram(cat_m+"_histograms_us", cams, weights_histo_us, dir_outputs_visu+cat_m) 
    
    cams_ecartees = [[3 * x / 2.2, y, 3*z/2.2] for x,y,z in cams ]
    weights_histo_m = [x / max(weights_method_m) for x in weights_method_m]
    show_cams_histogram(cat_m+"_histograms_m", cams_ecartees, weights_histo_m, dir_outputs_visu+cat_m)
    
    # En 2D
    histogram_circulaire(weights_method_us, weights_method_m, dir_outputs_visu, cat_us)
    histograms_2D(weights_method_us, weights_method_m, np.array(df_poids_both['label']), dir_outputs_visu, cat_us)
    ####### Métriques
    for name, metrique in dict_metriques.items():
        BVS[cat_us]['metriques'][name] = np.round(metrique(weights_method_us, weights_method_m),3)
    
## Valeur idéales 
ideal ={}; a = weights_method_us; ideal['PS'] = 1
for name, metrique in dict_metriques.items():
    ideal[name] = np.round(metrique(a, a),3)
  
# Metriques
metriques = list(BVS[cat_us]['metriques'].keys())
df_metriques = pd.DataFrame(columns=['categorie',] + metriques)
df_metriques.loc[len(df_metriques)] = ['ideal']+[ideal[m] for m in metriques]
for cat_us in list_categorie_us:
    df_metriques.loc[len(df_metriques)+1] = [cat_us]+[BVS[cat_us]['metriques'][m] for m in metriques]
    
df_metriques.to_csv(os.path.join(dir_outputs_csv, "metriques_modelnet40_user_study.csv"))    
df_metriques

### Sans Symétrique

Pour les symétriques, on somme les pids, car chaque précédement on a normalier pour que chaque objet ait un impact de 1, donc en tout on a que la somme de tous les poids == nb objet. Si on prend le poids max entre les symétriques, on perd cette égalité. Donc on prend la somme.

In [ ]:
_, df_exemple_sym = get_poids_from_BVS(BVS, cat_us, labels_us, data_us_cam); df_exemple_sym

In [ ]:
for cat_us in list_categroie_us_sym:
    cat_m = [k for k, v in match_ModelNet2US.items() if v == cat_us][0]
    _, df_poids_both_sym = get_poids_from_BVS(BVS, cat_us, labels_us, data_us_cam)
    # sauvegarde poids 
    df_poids_both_sym.to_csv(os.path.join(dir_outputs_csv, cat_us+"_poids_both_sym.csv"))
    # Poids des caméras pour les deux méthodes (exemple)
    weights_method_us = np.array(df_poids_both_sym['poids_us_norm'])
    weights_method_m = np.array(df_poids_both_sym['poids_modelnet_norm'])
    ####### Visualisation des poids 
    cams = [list(data_us_cam.loc[data_us_cam['label'] == l].iloc[0])[6:9] for l in df_poids_both_sym['label']]
    show_cams_histogram(cat_m+"_histograms_us_sym", cams, weights_method_us, dir_outputs_visu+cat_m) 
    
    cams_ecartees = [[3 * x / 2.2, y, 3*z/2.2] for x,y,z in cams ]
    show_cams_histogram(cat_m+"_histograms_m_sym", cams_ecartees, weights_method_m, dir_outputs_visu+cat_m)
    
    # En 2D
    histogram_circulaire(weights_method_us, weights_method_m, dir_outputs_visu, cat_us+"_sym_")
    histograms_2D(weights_method_us, weights_method_m, np.array(df_poids_both_sym['label']), dir_outputs_visu, cat_us+"_sym_")
    
    ####### Métriques
    for name, metrique in dict_metriques.items():
        BVS[cat_us]['metriques_sym'][name] = np.round(metrique(weights_method_us, weights_method_m),3)
    
## Valeur idéales 
ideal ={}; a = weights_method_us; ideal['PS'] = 1
for name, metrique in dict_metriques.items():
    ideal[name] = np.round(metrique(a, a),3)

# Metriques
metriques_sym = list(BVS[cat_us]['metriques_sym'].keys())
df_metriques_sym = pd.DataFrame(columns=['categorie',] + metriques_sym)
df_metriques_sym.loc[len(df_metriques_sym)] = ['ideal']+[ideal[m] for m in metriques]
for cat_us in list_categroie_us_sym:
    df_metriques_sym.loc[len(df_metriques_sym)+1] = [cat_us+"_sym"]+[BVS[cat_us]['metriques_sym'][m] for m in metriques_sym]
    
df_metriques_sym.to_csv(os.path.join(dir_outputs_csv, "metriques_modelnet40_user_study_sym.csv"))    
df_metriques_sym

## Visualisation globales des résultats

In [ ]:
# Score du terme C dans PS en fonction de leccart entre les caméras
cam_u = [1.56,1.56,0]; sss=1.5
print("egalite", get_ds2(coord_U= cam_u, coord_M=cam_u, sig=sss, epsilon=2),"-- 1 pas : ", get_ds2(coord_U= cam_u, coord_M=[1.1,1.56,1.1], sig=sss, epsilon=2), "-- 2 pas : ", get_ds2(coord_U= cam_u, coord_M=[0,1.56,1.56], sig=sss, epsilon=2), "-- 3 pas : ", get_ds2(coord_U= cam_u, coord_M=[-1.1,1.56,1.1], sig=sss, epsilon=2), "-- 4 pas : ", get_ds2(coord_U= cam_u, coord_M=[-1.56,1.56,0], sig=sss, epsilon=2))

In [ ]:
df_metriques

In [ ]:
df_metriques_sym